<a href="https://colab.research.google.com/github/amoyag/Bioquimica_Ing_Proteinas/blob/main/3-Scoring_function/clase3-score_alumnos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--NOTEBOOK_HEADER-->
*This notebook contains material from [PyRosetta](https://RosettaCommons.github.io/PyRosetta.notebooks);
content is available [on Github](https://github.com/RosettaCommons/PyRosetta.notebooks.git).*

# Working the Scoring function
Keywords: score function, ScoreFunction(), get_score_function(), set_weight(), show(), etable_atom_pair_energies(), Atom objects, get_hbonds(), nhbonds(), residue_hbonds()

## Init PyRosetta

In [ ]:
!pip install pyrosettacolabsetup
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()
import pyrosetta; pyrosetta.init()
from pyrosetta import *

#init()
#import os
#notebook_path = os.path.abspath("clase2-score.ipynb")


## Load pdb files

In [ ]:
pdb_file = "PATH_TO_FILE/5tj3.pdb"
clean_pdb_file = "PATH_TO_FILE/5tj3.clean.pdb"
pose = pose_from_pdb(pdb_file)
pose_clean = pose_from_pdb(clean_pdb_file)

## Rosetta Energy Score Functions


A basic function of Rosetta is calculating the energy or score of a biomolecule. This is important for inspecting the energies of a biomolecule at the whole protein, per-residue, and per-atom level.  

Rosetta has a standard energy function for all-atom calculations as well as several scoring functions for low-resolution protein representations. See https://www.ncbi.nlm.nih.gov/pubmed/28430426 for a review on the all-atom score functions.

You can also tailor an energy function by including scoring terms of your choice with custom weights.

To score a protein, you will begin by defining a score function using the `get_score_function(is_fullatom: bool)` method in the `pyrosetta.teaching` namespace. Specifying `True` will return the default `ref2015` all-atom energy function, while `False` will specify the default centroid score function.

Create a PyRosetta score function

In [ ]:
from pyrosetta.teaching import *

sfxn = get_score_function(True)

You can see the terms, weights, and energy method options by printing the score function:

```
print(sfxn)
```

In [ ]:
print(sfxn)

**Exercise. List the terms in the energy function and their relative weights**

**Hint:** look at the top line that starts with 'weights'

### Custom energy functions

You can also create a custom energy function that includes select terms. Typically, creating a whole new score function is unneccesary because the current one works well in most cases. However, tweaking the current energy function by reassigning weights and adding certain energy terms can be useful.

Here, we will make an example energy function with only the van der Waals attractive and repulsive terms, both with weights of 1. We need to use the `set_weight()`. Make a new `ScoreFunction` and set the weights accordingly. This is how we set the full-atom attractive (`fa_atr`) and the full-atom repulsive (`fa_rep`) terms.

```
sfxn2 = ScoreFunction()
sfxn2.set_weight(fa_atr, 1.0)
sfxn2.set_weight(fa_rep, 1.0)
```

In [ ]:
sfxn2 = ScoreFunction()
sfxn2.set_weight(fa_atr, 1.0)
sfxn2.set_weight(fa_rep, 1.0)

Lets compare the score of `pose_clean` using the full-atom `ScoreFunction` versus the `ScoreFunction` we made above using only the attractive and repulsive terms.

**Exercise. Print the total energy of `pose_clean` using  the `sfxn` score function.
Then, print the attractive and repulsive energy only using the custom `sfxn2` score function.**



### Energy Breakdown
Using the full-atom `ScoreFunction` `sfxn`, break the energy of `pose_clean` down into its individual pieces with the `sfxn.show()` method.

In [ ]:
sfxn.show(pose_clean)

**Exercise. Which are the three most dominant contributions, and what are their values? Is this what you would have expected? Why?** Note which terms are positive and negative

Unweighted, individual component energies of each residue in a structure are stored in the `Pose` object and can be accessed by the `energies()` method.

Note: The _backbone_ hydrogen-bonding terms for each residue are not available from the `Energies` object. You can get them by using EnergyMethodOptions. See http://www.pyrosetta.org/documentation#TOC-Hydrogen-Bonds-and-Hydrogen-Bond-Scoring.

**Exercise.** What are the total van der Waals, solvation, and hydrogen-bonding contributions of residue 24?

The van der Waals, solvation, and electrostatic terms are atom-atom pairwise energies calculated from a pre-tabulated lookup table, dependent upon the distance between the two atoms and their types. You can access this lookup table, called the `etable` directly to check these energy calculations on an atom-by-atom basis. Use the `pyrosetta.etable_atom_pair_energies` function which returns scores for attractive, repulsive, solvation and electrostatic potentials.

(Note that the `pyrosetta.etable_atom_pair_energies()` function requires `Atom` objects, not the `AtomID` objects we saw earlier.

You can access the `Atom` object for a residue with `residue.atom_index("")` and you can access to a residue with `pose.residue()`. For instance to access the Calpha of residue 15 of the pose `pose` you can use:

```
res15 = pose.residue(15)
res15_atomCA = res15.atom_index("CA")
```
For more info, look at the [documentation](https://graylab.jhu.edu/PyRosetta.documentation/pyrosetta.toolbox.atom_pair_energy.html?highlight=etable_atom_pair_energies#pyrosetta.toolbox.atom_pair_energy.etable_atom_pair_energies).)


In [ ]:
help(pyrosetta.etable_atom_pair_energies)

**Exercise. What are the attractive, repulsive, solvation, and electrostatic components between the nitrogen of residue 24 and the oxygen of residue 20?**

In [ ]:
res20 = pose.residue(20)
res20_atomCA = res20.atom_index("O")